# 上級編1

In [3]:
import numpy as np
import pandas as pd
from collections import Counter

In [7]:
# 生成多項式から生成行列を作る場合
# g = np.array([0, 0, 0, 1, 1, 0, 1])

# 巡回問3
g = np.array([0, 0, 1, 1, 1, 0, 1])

# 巡回問7
# g = np.array([0, 0, 1, 0, 1, 1, 1])

k = np.argmax(g) + 1
n = g.shape[0]
G = np.stack([g] * k, 0)
for i, row in enumerate(G):
    if i >= 1:
        G[i] = np.roll(G[i - 1], -1)
G = np.matrix(G, dtype="u1")
print(G)

[[0 0 1 1 1 0 1]
 [0 1 1 1 0 1 0]
 [1 1 1 0 1 0 0]]


In [8]:
# ブロック符号問1の生成行列
# G = np.matrix([
#     [1, 0, 1, 0, 1, 1],
#     [1, 1, 1, 1, 0, 1],
#     [0, 1, 1, 0, 0, 1]
# ], dtype="u1")

# 崩したハミング符号
# 1ビット以下の誤りのみ検出可能
# 完全符号
# G = np.matrix([
#     [1, 1, 0, 0, 1, 0, 0],
#     [1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

# ブロック問3の検査行列 (生成行列と等しい)
# G = np.matrix([
#     [0, 1, 1, 1, 1, 0, 0, 0],
#     [1, 1, 1, 0, 0, 1, 0, 0],
#     [1, 1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

k, n = G.shape
print(G)
k, n

[[0 0 1 1 1 0 1]
 [0 1 1 1 0 1 0]
 [1 1 1 0 1 0 0]]


(3, 7)

## 掃き出し法で組織符号の生成行列に変換

In [9]:
k = G.shape[0]

A = np.matrix(np.identity(k), dtype="u1")
G_src = G.copy()[:,:k]
I_k = np.identity(k)
# print(G_src)
for i in range(k):
#     print(f"掃き出し{i + 1}回目")
    col_i = np.array(G_src)[:,i]
    # 対角成分が0
    # 他の行と入れ替え
    if col_i[i] == 0:
        # 入れ替え対象の行番号
        j = np.where(col_i == 1)[0][-1]
        A_i = I_k.copy()
        A_i[i] = I_k[j].copy()
        A_i[j] = I_k[i].copy()
        A_i = np.matrix(A_i, dtype="u1")
        A = A_i * A % 2
        G_src = A_i * G_src % 2
#         print(G_src)
        col_i = np.array(G_src)[:,i]
    A_i =I_k.copy()
    A_i[:,i] = col_i
    A_i = np.matrix(A_i, dtype="u1")
    A = A_i * A % 2
    # print(f"A_i=\n{A_i}")
    # print(f"A=\n{A}")
    G_src = A_i * G_src % 2
#     print(G_src)

Tissue_G = A * G % 2
print(f"A=\n{A}")
print(f"G=\n{Tissue_G}")

A=
[[0 1 1]
 [1 1 0]
 [1 0 0]]
G=
[[1 0 0 1 1 1 0]
 [0 1 0 0 1 1 1]
 [0 0 1 1 1 0 1]]


## 標準アレイ

In [50]:
# 非零の全符号生成
all_src_ex0 = np.matrix([[int(j) for j in format(i, f"0{k}b")] for i in range(1, 2 ** k)], dtype="u1")
all_code_ex0 = all_src_ex0 * Tissue_G % 2

# 巡回問7. 手書き
# all_code_ex0_str = ["0010111", "0101110", "1011100", "0111001", "1110010", "1100101", "1001011"]
# all_code_ex0 = np.matrix([[int(j) for j in moji] for moji in all_code_ex0_str])

all_code_ex0_str = ["".join(str(i) for i in l) for l in all_code_ex0.tolist()]
all_code_ex0_str

['00011011',
 '00101101',
 '00110110',
 '01001110',
 '01010101',
 '01100011',
 '01111000',
 '10000111',
 '10011100',
 '10101010',
 '10110001',
 '11001001',
 '11010010',
 '11100100',
 '11111111']

In [90]:
# 1ビットエラー全パターン?
all_error_str = [format(2 ** i, f"0{n}b") for i in range(n)]
all_one_bit_error = [format(2 ** i, f"0{n}b") for i in range(n)]
custom_error_str = []
# print(all_one_bit_error)
# 適当に追加
# all_error_str += ["000011"]
# ブロック符号問3. (8, 4) 符号用
# custom_error_str = ["00000011", "00000101", "00001001", "00010001", "00100001", "01000001", "10000001"]
custom_error_str = ["00000011", "00000110", "00001010", "00010010", "00100010", "01000010", "10000010"]

# 巡回問7, 2bitバースト
# custom_error_str = ["0000011", "0000110", "0001100", "0011000", "0110000", "1100000", "1000001", "1111111"]

all_error_str = all_one_bit_error + custom_error_str

# ブロック符号第2問
# 起こりやすい誤りを勝手に定義
# all_error_str = ["000001", "000010", "000100", "001000", "010000", "000011", "000101"]
# all_error_str = ["000001", "000010", "001000", "010000", "100000", "101000", "100001"]

all_error = np.matrix([[int(j) for j in moji] for moji in all_error_str])
all_error_str

['00000001',
 '00000010',
 '00000100',
 '00001000',
 '00010000',
 '00100000',
 '01000000',
 '10000000',
 '00000011',
 '00000110',
 '00001010',
 '00010010',
 '00100010',
 '01000010',
 '10000010']

In [91]:
std_arr_matrix = [[np.matrix((e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
# std_arr_values = [["".join(str(k) for k in (e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
std_arr_values = [["".join(str(k) for k in j.tolist()[0]) for j in i] for i in std_arr_matrix]
# np.array(std_arr_values)
std_arr_df = pd.DataFrame(std_arr_values, columns=all_code_ex0_str, index=all_error_str)
std_arr_df

,00011011,00101101,00110110,01001110,01010101,01100011,01111000,10000111,10011100,10101010,10110001,11001001,11010010,11100100,11111111
00000001,00011010,00101100,00110111,01001111,01010100,01100010,01111001,10000110,10011101,10101011,10110000,11001000,11010011,11100101,11111110
00000010,00011001,00101111,00110100,01001100,01010111,01100001,01111010,10000101,10011110,10101000,10110011,11001011,11010000,11100110,11111101
00000100,00011111,00101001,00110010,01001010,01010001,01100111,01111100,10000011,10011000,10101110,10110101,11001101,11010110,11100000,11111011
00001000,00010011,00100101,00111110,01000110,01011101,01101011,01110000,10001111,10010100,10100010,10111001,11000001,11011010,11101100,11110111
00010000,00001011,00111101,00100110,01011110,01000101,01110011,01101000,10010111,10001100,10111010,10100001,11011001,11000010,11110100,11101111
00100000,00111011,00001101,00010110,01101110,01110101,01000011,01011000,10100111,10111100,10001010,10010001,11101001,11110010,11000100,11011111
01000000,01011011,01101101,01110110,00001110,00010101,00100011,00111000,11000111,11011100,11101010,11110001,10001001,10010010,10100100,10111111
10000000,10011011,10101101,10110110,11001110,11010101,11100011,11111000,00000111,00011100,00101010,00110001,01001001,01010010,01100100,01111111
00000011,00011000,00101110,00110101,01001101,01010110,01100000,01111011,10000100,10011111,10101001,10110010,11001010,11010001,11100111,11111100
00000110,00011101,00101011,00110000,01001000,01010011,01100101,01111110,10000001,10011010,10101100,10110111,11001111,11010100,11100010,11111001


In [92]:
# 重複がないか確認
table_set = {"000000"}
table_set |= set(std_arr_df.values.ravel())
table_set |= set(all_code_ex0_str)
table_set |= set(all_error_str)
print(len(table_set))

256


## 各受信語のシンドローム計算

In [38]:
# パリティ生成部分
P = Tissue_G[:, k:]
# print(P)
# 検査行列
H = np.concatenate([P.T, np.identity(n - k, dtype=np.uint8)], 1)
H

matrix([[1, 1, 1, 1, 0, 0],
        [0, 1, 1, 0, 1, 0],
        [0, 0, 1, 0, 0, 1]], dtype=uint8)

In [39]:
syndrome_values = [["".join(str(k) for k in (j * H.T % 2).tolist()[0]) for j in i] for i in std_arr_matrix]
syndrome_df = pd.DataFrame(syndrome_values, columns=all_code_ex0_str, index=all_error_str)
syndrome_df

,001111,010110,011001,100100,101011,110010,111101
000001,001,001,001,001,001,001,001
000010,010,010,010,010,010,010,010
001000,111,111,111,111,111,111,111
010000,110,110,110,110,110,110,110
100000,100,100,100,100,100,100,100
101000,011,011,011,011,011,011,011
100001,101,101,101,101,101,101,101


## 最小ハミング距離

In [44]:
# G = Tissue_G
G = np.matrix([
    [1, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0, 1, 1]
])

k, n = G.shape
print(G)
print(k, n)

all_src_ex0 = [np.matrix([int(j) for j in format(i, f"0{k}b")], dtype=np.uint8) for i in range(1, 1 << k)]
all_code_ex0 = [i * G % 2 for i in all_src_ex0]
code_df = pd.DataFrame(
    ["".join([str(j) for j in i.tolist()[0]]) for i in all_code_ex0],
    index=["".join([str(j) for j in i.tolist()[0]]) for i in all_src_ex0]
)
all_code_ex0_hamming_dist = [i.sum() for i in all_code_ex0]
hamming_dist_min = min(all_code_ex0_hamming_dist)
print(f"最小ハミング距離: {hamming_dist_min}")
# all_code_ex0
code_df

[[1 0 0 0 0 1 1 1]
 [0 1 0 0 1 1 1 0]
 [0 0 1 0 1 1 0 1]
 [0 0 0 1 1 0 1 1]]
4 8
最小ハミング距離: 4


,0
0001,00011011
0010,00101101
0011,00110110
0100,01001110
0101,01010101
0110,01100011
0111,01111000
1000,10000111
1001,10011100
1010,10101010


In [93]:
A_i_dic = Counter(all_code_ex0_hamming_dist)
print(A_i_dic)
p = 0.01
P_u_E = 0
for i, A_i in A_i_dic.items():
    P_u_E += A_i * p ** i * (1 - p) ** (n - i)
P_u_E

Counter({4: 14, 8: 1})


1.344834415e-07